# Introduction

**SUMMARY**

The purpose of this report is to analyze health inequities in the US by county and how those inequities influence the degree of impact of the COVID-19 pandemic.

____

**STAKEHOLDERS**

Federal, state, and local governments and epidemiologists can use this analysis to shape public health policy and pandemic response plans. The goal is to learn from COVID-19 to prevent the loss that occurred in 2020-2021 from happening again.

____

**DATA SOURCES**

- [CDC’s Social Vulnerability Index](https://www.atsdr.cdc.gov/placeandhealth/svi/documentation/SVI_documentation_2018.html)

- [CDC's Vaccine Hesitancy](https://data.cdc.gov/Vaccinations/Vaccine-Hesitancy-for-COVID-19-County-and-local-es/q9mh-h2tw)

- [The University of Wisconsin Population Health Institute's County Health Rankings](https://www.countyhealthrankings.org/explore-health-rankings/measures-data-sources) 

- [New York Times COVID Case and Death Counts](https://github.com/nytimes/covid-19-data)


____

**METHODS**

Classification model which predicts a given county’s degree of impact of the pandemic (high impact, average impact, low impact). Feature importance and coefficients will be analyzed to recommend what features put communities at greater risk in order to design policies to address the underlying issues.

____

**LOAD PACKAGES**

In [1]:
# style notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

In [2]:
#import standard packages
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

#import viz packages
import matplotlib.pyplot as plt
%matplotlib inline
#plt.rcParams['figure.figsize'] = (15,5) #setting figures to timeseries-friendly
import plotly.express as px
import plotly.graph_objects as go

#import time series tools from statsmodels
import statsmodels.tsa.api as tsa
import statsmodels
print(f'Statsmodels version = {statsmodels.__version__}')

Statsmodels version = 0.12.0


# Data Load

## Compress

Data files downloaded from data.cdc.gov, Univ of Wisconsin Public Health Institute, and the New York Times are too large to push to github in their raw form, so the first step is to compress the files.

**Social Vulnerability Index:**

In [3]:
svi = pd.read_csv('data/SVI2018_US_COUNTY.csv')
svi.head()

,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV,MP_POV,EP_UNEMP,MP_UNEMP,EP_PCI,MP_PCI,EP_NOHSDP,MP_NOHSDP,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_MINRTY,MP_MINRTY,EP_LIMENG,MP_LIMENG,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,71,12398,439,-999,-999,-999,-999,-999,-999,3669,426,7083,25,9318,14,6280,495,1330,285.0,34397,145.0,755,209.5,67,37.1,7770,431,264,77.1,763,160,654,142,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,13.8,1.6,18.0,0.1,23.7,0.0,16.1,1.3,10.7,2.3,87.5,0.4,2.1,0.6,0.3,0.2,38.8,2.2,2.1,0.6,6.2,1.3,1.7,0.4,-999.0000,-999.0000,-999.0000,0.5922,-999.0000,-999.0000,0.4893,0.6826,0.5610,0.8383,2.5712,0.8758,0.9917,0.7740,1.7657,0.9268,0.0551,0.9869,0.5498,0.5788,0.4126,2.5832,0.5409,-999.0000,-999.0000,-999,-999,-999,0,-999,0,0,0,0,0,1,0,1,0,1,0,0,0,1,-999,4160,588,10.6,1.5,32290
1,1,ALABAMA,AL,Autauga,1001,"Autauga County, Alabama",594.443459,55200,0,23315,71,21115,383,8422,1137,1065,257,29372,2306,4204,475,8050,75,13369,32,10465,729,1586,319.9,13788,59.0,426,205.9,886,308.7,4279,469,299,142.3,1191,272,546,161,15.4,2.1,4.2,1.0,29372.0,2306.0,11.3,1.3,14.6,0.1,24.2,0.1,19.3,1.3,7.5,1.5,25.0,0.1,0.8,0.4,3.8,1.3,18.4,2.0,1.4,0.7,5.6,1.3,1.0,0.3,0.5401,0.2745,0.2860,0.4397,1.5403,0.3631,0.1850,0.7529,0.7905,0.3792,2.1076,0.5810,0.6336,0.5113,1.1449,0.5947,0.6017,0.7408,0.2964,0.4846,0.1525,2.2760,0.3741,7.0688,0.4354,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3875,508,7.1,0.9,37301
2,1,ALABAMA,AL,Blount,1009,"Blount County, Alabama",644.830460,57645,0,24222,55,20600,396,8220,992,909,193,22656,905,7861,727,10233,91,13468,53,8114,592,1437,267.2,7413,229.0,934,239.3,211,104.2,6108,476,339,130.7,856,201,543,117,14.4,1.7,4.1,0.8,22656.0,905.0,19.8,1.8,17.8,0.2,23.4,0.1,14.2,1.0,7.0,1.3,12.9,0.4,1.7,0.4,0.9,0.4,25.2,2.0,1.6,0.6,4.2,1.0,0.9,0.2,0.4723,0.2611,0.7561,0.8405,2.3300,0.6143,0.4715,0.6406,0.3763,0.2961,1.7845,0.3187,0.4206,0.7170,1.1376,0.5915,0.1512,0.8816,0.3703,0.2420,0.1165,1.7616,0.1741,7.0137,0.4242,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6303,732,11.0,1.3,40036
3,1,ALABAMA,AL,Butler,1013,"Butler County, Alabama",776.838201,20025,0,10026,51,6708,274,4640,521,567,147,20430,1258,2141,268,3806,21,4566,88,3492,370,704,143.9,9641,22.0,93,137.4,134,47.4,2625,212,119,57.7,520,102,322,88,23.5,2.6,6.7,1.7,20430.0,1258.0,15.4,1.9,19.0,0.1,22.8,0.4,17.7,1.9,10.5,2.1,48.1,0.1,0.5,0.7,1.3,0.5,26.2,2.1,1.8,0.9,7.8,1.5,1.6,0.4,0.8860,0.6968,0.8879,0.6753,3.1460,0.8455,0.5928,0.5578,0.6845,0.8185,2.6536,0.9077,0.8711,0.3582,1.2293,0.6447,0.2416,0.8940,0.4457,0.7685,0.3792,2.7290,0.6259,9.7579,0.8653,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2005,340,10.2,1.7,17280
4,1,ALABAMA,AL,Calhoun,1015,"Calhoun County, Alabama",605.867251,115098,0,53682,184,45033,683,20819,1317,4628,526,24706,758,12620,766,19386,119,25196,66,23598,1086,4701,464.0,31675,34.0,1076,250.2,1990,303.0,7904,546,772,206.2,2599,331,3112,436,18.6,1.2,8.8,1.0,24706.0,758.0,15.9,1.0,16.8,0.1,21.9,0.1,20.8,0.9,10.4,1.0,27.5,0.0,1.0,0.2,3.7,0.6,14.7,1.0,1.7,0.5,5.8,0.7,2.7,0.4,0.7322,0.8850,0.6076,0.6988,2.9236,0.7866,0.3664,0.4323,0.8

In [4]:
svi.to_csv('data_compressed/SVI2018_US_COUNTY.csv.gz', compression='gzip', index=False)

In [5]:
svi = pd.read_csv('data_compressed/SVI2018_US_COUNTY.csv.gz', compression='gzip')
svi.head()

,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV,MP_POV,EP_UNEMP,MP_UNEMP,EP_PCI,MP_PCI,EP_NOHSDP,MP_NOHSDP,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_MINRTY,MP_MINRTY,EP_LIMENG,MP_LIMENG,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,71,12398,439,-999,-999,-999,-999,-999,-999,3669,426,7083,25,9318,14,6280,495,1330,285.0,34397,145.0,755,209.5,67,37.1,7770,431,264,77.1,763,160,654,142,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,13.8,1.6,18.0,0.1,23.7,0.0,16.1,1.3,10.7,2.3,87.5,0.4,2.1,0.6,0.3,0.2,38.8,2.2,2.1,0.6,6.2,1.3,1.7,0.4,-999.0000,-999.0000,-999.0000,0.5922,-999.0000,-999.0000,0.4893,0.6826,0.5610,0.8383,2.5712,0.8758,0.9917,0.7740,1.7657,0.9268,0.0551,0.9869,0.5498,0.5788,0.4126,2.5832,0.5409,-999.0000,-999.0000,-999,-999,-999,0,-999,0,0,0,0,0,1,0,1,0,1,0,0,0,1,-999,4160,588,10.6,1.5,32290
1,1,ALABAMA,AL,Autauga,1001,"Autauga County, Alabama",594.443459,55200,0,23315,71,21115,383,8422,1137,1065,257,29372,2306,4204,475,8050,75,13369,32,10465,729,1586,319.9,13788,59.0,426,205.9,886,308.7,4279,469,299,142.3,1191,272,546,161,15.4,2.1,4.2,1.0,29372.0,2306.0,11.3,1.3,14.6,0.1,24.2,0.1,19.3,1.3,7.5,1.5,25.0,0.1,0.8,0.4,3.8,1.3,18.4,2.0,1.4,0.7,5.6,1.3,1.0,0.3,0.5401,0.2745,0.2860,0.4397,1.5403,0.3631,0.1850,0.7529,0.7905,0.3792,2.1076,0.5810,0.6336,0.5113,1.1449,0.5947,0.6017,0.7408,0.2964,0.4846,0.1525,2.2760,0.3741,7.0688,0.4354,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3875,508,7.1,0.9,37301
2,1,ALABAMA,AL,Blount,1009,"Blount County, Alabama",644.830460,57645,0,24222,55,20600,396,8220,992,909,193,22656,905,7861,727,10233,91,13468,53,8114,592,1437,267.2,7413,229.0,934,239.3,211,104.2,6108,476,339,130.7,856,201,543,117,14.4,1.7,4.1,0.8,22656.0,905.0,19.8,1.8,17.8,0.2,23.4,0.1,14.2,1.0,7.0,1.3,12.9,0.4,1.7,0.4,0.9,0.4,25.2,2.0,1.6,0.6,4.2,1.0,0.9,0.2,0.4723,0.2611,0.7561,0.8405,2.3300,0.6143,0.4715,0.6406,0.3763,0.2961,1.7845,0.3187,0.4206,0.7170,1.1376,0.5915,0.1512,0.8816,0.3703,0.2420,0.1165,1.7616,0.1741,7.0137,0.4242,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6303,732,11.0,1.3,40036
3,1,ALABAMA,AL,Butler,1013,"Butler County, Alabama",776.838201,20025,0,10026,51,6708,274,4640,521,567,147,20430,1258,2141,268,3806,21,4566,88,3492,370,704,143.9,9641,22.0,93,137.4,134,47.4,2625,212,119,57.7,520,102,322,88,23.5,2.6,6.7,1.7,20430.0,1258.0,15.4,1.9,19.0,0.1,22.8,0.4,17.7,1.9,10.5,2.1,48.1,0.1,0.5,0.7,1.3,0.5,26.2,2.1,1.8,0.9,7.8,1.5,1.6,0.4,0.8860,0.6968,0.8879,0.6753,3.1460,0.8455,0.5928,0.5578,0.6845,0.8185,2.6536,0.9077,0.8711,0.3582,1.2293,0.6447,0.2416,0.8940,0.4457,0.7685,0.3792,2.7290,0.6259,9.7579,0.8653,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2005,340,10.2,1.7,17280
4,1,ALABAMA,AL,Calhoun,1015,"Calhoun County, Alabama",605.867251,115098,0,53682,184,45033,683,20819,1317,4628,526,24706,758,12620,766,19386,119,25196,66,23598,1086,4701,464.0,31675,34.0,1076,250.2,1990,303.0,7904,546,772,206.2,2599,331,3112,436,18.6,1.2,8.8,1.0,24706.0,758.0,15.9,1.0,16.8,0.1,21.9,0.1,20.8,0.9,10.4,1.0,27.5,0.0,1.0,0.2,3.7,0.6,14.7,1.0,1.7,0.5,5.8,0.7,2.7,0.4,0.7322,0.8850,0.6076,0.6988,2.9236,0.7866,0.3664,0.4323,0.8

**Vaccine Hesitancy:**

In [6]:
vax_hesitancy = pd.read_csv('data/Vaccine_Hesitancy_for_COVID-19__County_and_local_estimates.csv')
vax_hesitancy.head()

,FIPS Code,County Name,State,Estimated hesitant,Estimated hesitant or unsure,Estimated strongly hesitant,Social Vulnerability Index (SVI),SVI Category,CVAC level of concern for vaccination rollout,CVAC Level Of Concern,Percent adults fully vaccinated against COVID-19 (as of 6/10/21),Percent Hispanic,Percent non-Hispanic American Indian/Alaska Native,Percent non-Hispanic Asian,Percent non-Hispanic Black,Percent non-Hispanic Native Hawaiian/Pacific Islander,Percent non-Hispanic White,Geographical Point,State Code,County Boundary,State Boundary
0,1123,"Tallapoosa County, Alabama",ALABAMA,0.1806,0.2400,0.1383,0.89,Very High Vulnerability,0.64,High Concern,0.305,0.0242,0.0022,0.0036,0.2697,0.0000,0.6887,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-85.841259 33.104456, -85.8409...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
1,1121,"Talladega County, Alabama",ALABAMA,0.1783,0.2350,0.1368,0.87,Very High Vulnerability,0.84,Very High Concern,0.265,0.0229,0.0043,0.0061,0.3237,0.0003,0.6263,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-86.303069 33.46316, -86.30306...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
2,1131,"Wilcox County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.93,Very High Vulnerability,0.94,Very High Concern,0.394,0.0053,0.0009,0.0003,0.6938,0.0000,0.2684,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-87.52534299999999 32.132773, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
3,1129,"Washington County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.73,High Vulnerability,0.82,Very High Concern,0.308,0.0146,0.0731,0.0025,0.2354,0.0000,0.6495,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-88.45317899999999 31.505388, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
4,1133,"Winston County, Alabama",ALABAMA,0.1805,0.2313,0.1379,0.70,High Vulnerability,0.80,High Concern,0.163,0.0315,0.0034,0.0016,0.0073,0.0005,0.9370,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-87.63656399999999 34.120908, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."


In [7]:
vax_hesitancy.to_csv('data_compressed/Vaccine_Hesitancy_for_COVID-19__County_and_local_estimates.csv.gz', 
                    compression='gzip', index=False)

In [8]:
vax_hesitancy = pd.read_csv('data_compressed/Vaccine_Hesitancy_for_COVID-19__County_and_local_estimates.csv.gz')
vax_hesitancy.head()

,FIPS Code,County Name,State,Estimated hesitant,Estimated hesitant or unsure,Estimated strongly hesitant,Social Vulnerability Index (SVI),SVI Category,CVAC level of concern for vaccination rollout,CVAC Level Of Concern,Percent adults fully vaccinated against COVID-19 (as of 6/10/21),Percent Hispanic,Percent non-Hispanic American Indian/Alaska Native,Percent non-Hispanic Asian,Percent non-Hispanic Black,Percent non-Hispanic Native Hawaiian/Pacific Islander,Percent non-Hispanic White,Geographical Point,State Code,County Boundary,State Boundary
0,1123,"Tallapoosa County, Alabama",ALABAMA,0.1806,0.2400,0.1383,0.89,Very High Vulnerability,0.64,High Concern,0.305,0.0242,0.0022,0.0036,0.2697,0.0000,0.6887,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-85.841259 33.104456, -85.8409...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
1,1121,"Talladega County, Alabama",ALABAMA,0.1783,0.2350,0.1368,0.87,Very High Vulnerability,0.84,Very High Concern,0.265,0.0229,0.0043,0.0061,0.3237,0.0003,0.6263,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-86.303069 33.46316, -86.30306...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
2,1131,"Wilcox County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.93,Very High Vulnerability,0.94,Very High Concern,0.394,0.0053,0.0009,0.0003,0.6938,0.0000,0.2684,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-87.52534299999999 32.132773, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
3,1129,"Washington County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.73,High Vulnerability,0.82,Very High Concern,0.308,0.0146,0.0731,0.0025,0.2354,0.0000,0.6495,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-88.45317899999999 31.505388, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
4,1133,"Winston County, Alabama",ALABAMA,0.1805,0.2313,0.1379,0.70,High Vulnerability,0.80,High Concern,0.163,0.0315,0.0034,0.0016,0.0073,0.0005,0.9370,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-87.63656399999999 34.120908, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."


**County Health Rankings:**

In [9]:
county_health_rankings = pd.read_csv('data/countyhealthrankings-analytic_data2021.csv', header=[0,1])
county_health_rankings.head()

,State FIPS Code,County FIPS Code,5-digit FIPS Code,State Abbreviation,Name,Release Year,County Ranked (Yes=1/No=0),Premature death raw value,Premature death numerator,Premature death denominator,Premature death CI low,Premature death CI high,Premature death flag (0 = No Flag/1=Unreliable/2=Suppressed),Premature death (AIAN),Premature death CI low (AIAN),Premature death CI high (AIAN),Premature death flag (AIAN) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (Asian/Pacific Islander),Premature death CI low (Asian/Pacific Islander),Premature death CI high (Asian/Pacific Islander),Premature death flag (Asian/Pacific Islander) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (Black),Premature death CI low (Black),Premature death CI high (Black),Premature death flag (Black) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (Hispanic),Premature death CI low (Hispanic),Premature death CI high (Hispanic),Premature death flag (Hispanic) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (White),Premature death CI low (White),Premature death CI high (White),Premature death flag (White) (. = No Flag/1=Unreliable/2=Suppressed),Poor or fair health raw value,Poor or fair health numerator,Poor or fair health denominator,Poor or fair health CI low,Poor or fair health CI high,Poor physical health days raw value,Poor physical health days numerator,Poor physical health days denominator,Poor physical health days CI low,Poor physical health days CI high,Poor mental health days raw value,Poor mental health days numerator,Poor mental health days denominator,Poor mental health days CI low,Poor mental health days CI high,Low birthweight raw value,Low birthweight numerator,Low birthweight denominator,Low birthweight CI low,Low birthweight CI high,LBW unreliable indicator (Unreliable = Numerator < 20 or relative standard error > 20%),Low birthweight (AIAN),Low birthweight CI low (AIAN),Low birthweight CI high (AIAN),Low birthweight (Asian/Pacific Islander),Low birthweight CI low (Asian/Pacific Islander),Low birthweight CI high (Asian/Pacific Islander),Low birthweight (Black),Low birthweight CI low (Black),Low birthweight CI high (Black),Low birthweight (Hispanic),Low birthweight CI low (Hispanic),Low birthweight CI high (Hispanic),Low birthweight (White),Low birthweight CI low (White),Low birthweight CI high (White),Adult smoking raw value,Adult smoking numerator,Adult smoking denominator,Adult smoking CI low,Adult smoking CI high,Adult obesity raw value,Adult obesity numerator,Adult obesity denominator,Adult obesity CI low,Adult obesity CI high,Food environment index raw value,Food environment index numerator,Food environment index denominator,Food environment index CI low,Food environment index CI high,Physical inactivity raw value,Physical inactivity numerator,Physical inactivity denominator,Physical inactivity CI low,Physical inactivity CI high,Access to exercise opportunities raw value,Access to exercise opportunities numerator,Access to exercise opportunities denominator,Access to exercise opportunities CI low,Access to exercise opportunities CI high,Excessive drinking raw value,Excessive drinking numerator,Excessive drinking denominator,Excessive drinking CI low,Excessive drinking CI high,Alcohol-impaired driving deaths raw value,Alcohol-impaired driving deaths numerator,Alcohol-impaired driving deaths denominator,Alcohol-impaired driving deaths CI low,Alcohol-impaired driving deaths CI high,Sexually transmitted infections raw value,Sexually transmitted infections numerator,Sexually transmitted infections denominator,Sexually transmitted infections CI low,Sexually transmitted infections CI high,Teen births raw value,Teen births numerator,Teen births denominator,Teen births CI low,Teen births CI high,Teen births (AIAN),Teen births CI low (AIAN),Teen births CI high (AIAN),Teen births (Asian/Pacific Islander),Teen births CI low (Asian/Pacific Islander),Teen births CI high (Asian/Pacific Islander),Teen births (Black),Teen b

In [10]:
county_health_rankings.to_csv('data_compressed/countyhealthrankings-analytic_data2021.csv.gz', compression='gzip',
                             index=False)

In [11]:
county_health_rankings = pd.read_csv('data_compressed/countyhealthrankings-analytic_data2021.csv.gz', 
                                     compression='gzip', header=[0,1])
county_health_rankings.head()

,State FIPS Code,County FIPS Code,5-digit FIPS Code,State Abbreviation,Name,Release Year,County Ranked (Yes=1/No=0),Premature death raw value,Premature death numerator,Premature death denominator,Premature death CI low,Premature death CI high,Premature death flag (0 = No Flag/1=Unreliable/2=Suppressed),Premature death (AIAN),Premature death CI low (AIAN),Premature death CI high (AIAN),Premature death flag (AIAN) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (Asian/Pacific Islander),Premature death CI low (Asian/Pacific Islander),Premature death CI high (Asian/Pacific Islander),Premature death flag (Asian/Pacific Islander) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (Black),Premature death CI low (Black),Premature death CI high (Black),Premature death flag (Black) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (Hispanic),Premature death CI low (Hispanic),Premature death CI high (Hispanic),Premature death flag (Hispanic) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (White),Premature death CI low (White),Premature death CI high (White),Premature death flag (White) (. = No Flag/1=Unreliable/2=Suppressed),Poor or fair health raw value,Poor or fair health numerator,Poor or fair health denominator,Poor or fair health CI low,Poor or fair health CI high,Poor physical health days raw value,Poor physical health days numerator,Poor physical health days denominator,Poor physical health days CI low,Poor physical health days CI high,Poor mental health days raw value,Poor mental health days numerator,Poor mental health days denominator,Poor mental health days CI low,Poor mental health days CI high,Low birthweight raw value,Low birthweight numerator,Low birthweight denominator,Low birthweight CI low,Low birthweight CI high,LBW unreliable indicator (Unreliable = Numerator < 20 or relative standard error > 20%),Low birthweight (AIAN),Low birthweight CI low (AIAN),Low birthweight CI high (AIAN),Low birthweight (Asian/Pacific Islander),Low birthweight CI low (Asian/Pacific Islander),Low birthweight CI high (Asian/Pacific Islander),Low birthweight (Black),Low birthweight CI low (Black),Low birthweight CI high (Black),Low birthweight (Hispanic),Low birthweight CI low (Hispanic),Low birthweight CI high (Hispanic),Low birthweight (White),Low birthweight CI low (White),Low birthweight CI high (White),Adult smoking raw value,Adult smoking numerator,Adult smoking denominator,Adult smoking CI low,Adult smoking CI high,Adult obesity raw value,Adult obesity numerator,Adult obesity denominator,Adult obesity CI low,Adult obesity CI high,Food environment index raw value,Food environment index numerator,Food environment index denominator,Food environment index CI low,Food environment index CI high,Physical inactivity raw value,Physical inactivity numerator,Physical inactivity denominator,Physical inactivity CI low,Physical inactivity CI high,Access to exercise opportunities raw value,Access to exercise opportunities numerator,Access to exercise opportunities denominator,Access to exercise opportunities CI low,Access to exercise opportunities CI high,Excessive drinking raw value,Excessive drinking numerator,Excessive drinking denominator,Excessive drinking CI low,Excessive drinking CI high,Alcohol-impaired driving deaths raw value,Alcohol-impaired driving deaths numerator,Alcohol-impaired driving deaths denominator,Alcohol-impaired driving deaths CI low,Alcohol-impaired driving deaths CI high,Sexually transmitted infections raw value,Sexually transmitted infections numerator,Sexually transmitted infections denominator,Sexually transmitted infections CI low,Sexually transmitted infections CI high,Teen births raw value,Teen births numerator,Teen births denominator,Teen births CI low,Teen births CI high,Teen births (AIAN),Teen births CI low (AIAN),Teen births CI high (AIAN),Teen births (Asian/Pacific Islander),Teen births CI low (Asian/Pacific Islander),Teen births CI high (Asian/Pacific Islander),Teen births (Black),Teen b

**NYT COVID-19 Case & Death Counts:**

In [12]:
nyt_cases_deaths = pd.read_csv('data/NYT-covid-cases-deaths.csv')
nyt_cases_deaths

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0
...,...,...,...,...,...,...
1462594,2021-06-27,Sweetwater,Wyoming,56037.0,4729,40.0
1462595,2021-06-27,Teton,Wyoming,56039.0,3801,11.0
1462596,2021-06-27,Uinta,Wyoming,56041.0,2315,13.0
1462597,2021-06-27,Washakie,Wyoming,56043.0,925,26.0


In [13]:
nyt_cases_deaths.to_csv('data_compressed/NYT-covid-cases-deaths.csv.gz', index=False, compression='gzip')

In [14]:
nyt_cases_deaths = pd.read_csv('data_compressed/NYT-covid-cases-deaths.csv.gz', compression='gzip')
nyt_cases_deaths

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0
...,...,...,...,...,...,...
1462594,2021-06-27,Sweetwater,Wyoming,56037.0,4729,40.0
1462595,2021-06-27,Teton,Wyoming,56039.0,3801,11.0
1462596,2021-06-27,Uinta,Wyoming,56041.0,2315,13.0
1462597,2021-06-27,Washakie,Wyoming,56043.0,925,26.0


## Explore Data Sources

Each file will now be explored and basic preprocessing steps like indexing, relabeling, setting data types, and dropping unnecessary features will be completed.

___ 


**SOCIAL VULNERABILITY INDEX**

Source: [CDC SVI 2018 Documentation](https://www.atsdr.cdc.gov/placeandhealth/svi/documentation/SVI_documentation_2018.html)

The degree to which a community exhibits certain social conditions, including high poverty, low percentage of vehicle access, or crowded households, may affect that community’s ability to prevent human suffering and financial loss in the event of disaster. These factors describe a community’s *social vulnerability.*

The Social Vulnerability Index (SVI) was created by the CDC to help public health officials and emergency response planners identify and map the communities that will most likely need support before, during, and after a hazardous event. While usually ranked at the US Census tract level, SVI 2018, used in this analysis, ranks at the county level. Each county received a ranking for each of the 15 social factors, for each of the four themes, as well as one overall ranking. 

Variables used:
<img src='images/SVI_variables.jpg'>

In [15]:
svi.head()

,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV,MP_POV,EP_UNEMP,MP_UNEMP,EP_PCI,MP_PCI,EP_NOHSDP,MP_NOHSDP,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_MINRTY,MP_MINRTY,EP_LIMENG,MP_LIMENG,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,71,12398,439,-999,-999,-999,-999,-999,-999,3669,426,7083,25,9318,14,6280,495,1330,285.0,34397,145.0,755,209.5,67,37.1,7770,431,264,77.1,763,160,654,142,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,13.8,1.6,18.0,0.1,23.7,0.0,16.1,1.3,10.7,2.3,87.5,0.4,2.1,0.6,0.3,0.2,38.8,2.2,2.1,0.6,6.2,1.3,1.7,0.4,-999.0000,-999.0000,-999.0000,0.5922,-999.0000,-999.0000,0.4893,0.6826,0.5610,0.8383,2.5712,0.8758,0.9917,0.7740,1.7657,0.9268,0.0551,0.9869,0.5498,0.5788,0.4126,2.5832,0.5409,-999.0000,-999.0000,-999,-999,-999,0,-999,0,0,0,0,0,1,0,1,0,1,0,0,0,1,-999,4160,588,10.6,1.5,32290
1,1,ALABAMA,AL,Autauga,1001,"Autauga County, Alabama",594.443459,55200,0,23315,71,21115,383,8422,1137,1065,257,29372,2306,4204,475,8050,75,13369,32,10465,729,1586,319.9,13788,59.0,426,205.9,886,308.7,4279,469,299,142.3,1191,272,546,161,15.4,2.1,4.2,1.0,29372.0,2306.0,11.3,1.3,14.6,0.1,24.2,0.1,19.3,1.3,7.5,1.5,25.0,0.1,0.8,0.4,3.8,1.3,18.4,2.0,1.4,0.7,5.6,1.3,1.0,0.3,0.5401,0.2745,0.2860,0.4397,1.5403,0.3631,0.1850,0.7529,0.7905,0.3792,2.1076,0.5810,0.6336,0.5113,1.1449,0.5947,0.6017,0.7408,0.2964,0.4846,0.1525,2.2760,0.3741,7.0688,0.4354,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3875,508,7.1,0.9,37301
2,1,ALABAMA,AL,Blount,1009,"Blount County, Alabama",644.830460,57645,0,24222,55,20600,396,8220,992,909,193,22656,905,7861,727,10233,91,13468,53,8114,592,1437,267.2,7413,229.0,934,239.3,211,104.2,6108,476,339,130.7,856,201,543,117,14.4,1.7,4.1,0.8,22656.0,905.0,19.8,1.8,17.8,0.2,23.4,0.1,14.2,1.0,7.0,1.3,12.9,0.4,1.7,0.4,0.9,0.4,25.2,2.0,1.6,0.6,4.2,1.0,0.9,0.2,0.4723,0.2611,0.7561,0.8405,2.3300,0.6143,0.4715,0.6406,0.3763,0.2961,1.7845,0.3187,0.4206,0.7170,1.1376,0.5915,0.1512,0.8816,0.3703,0.2420,0.1165,1.7616,0.1741,7.0137,0.4242,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6303,732,11.0,1.3,40036
3,1,ALABAMA,AL,Butler,1013,"Butler County, Alabama",776.838201,20025,0,10026,51,6708,274,4640,521,567,147,20430,1258,2141,268,3806,21,4566,88,3492,370,704,143.9,9641,22.0,93,137.4,134,47.4,2625,212,119,57.7,520,102,322,88,23.5,2.6,6.7,1.7,20430.0,1258.0,15.4,1.9,19.0,0.1,22.8,0.4,17.7,1.9,10.5,2.1,48.1,0.1,0.5,0.7,1.3,0.5,26.2,2.1,1.8,0.9,7.8,1.5,1.6,0.4,0.8860,0.6968,0.8879,0.6753,3.1460,0.8455,0.5928,0.5578,0.6845,0.8185,2.6536,0.9077,0.8711,0.3582,1.2293,0.6447,0.2416,0.8940,0.4457,0.7685,0.3792,2.7290,0.6259,9.7579,0.8653,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2005,340,10.2,1.7,17280
4,1,ALABAMA,AL,Calhoun,1015,"Calhoun County, Alabama",605.867251,115098,0,53682,184,45033,683,20819,1317,4628,526,24706,758,12620,766,19386,119,25196,66,23598,1086,4701,464.0,31675,34.0,1076,250.2,1990,303.0,7904,546,772,206.2,2599,331,3112,436,18.6,1.2,8.8,1.0,24706.0,758.0,15.9,1.0,16.8,0.1,21.9,0.1,20.8,0.9,10.4,1.0,27.5,0.0,1.0,0.2,3.7,0.6,14.7,1.0,1.7,0.5,5.8,0.7,2.7,0.4,0.7322,0.8850,0.6076,0.6988,2.9236,0.7866,0.3664,0.4323,0.8

In [16]:
svi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Columns: 123 entries, ST to E_DAYPOP
dtypes: float64(63), int64(56), object(4)
memory usage: 2.9+ MB


In [17]:
# check for duplicate fips value
svi.FIPS.value_counts().sort_values()

47103    1
47071    1
47089    1
47101    1
47099    1
        ..
47111    1
47109    1
47107    1
47105    1
8115     1
Name: FIPS, Length: 3142, dtype: int64

In [18]:
# set fips code as index
svi.set_index('FIPS', inplace=True)

In [19]:
list(svi.columns)

['ST',
 'STATE',
 'ST_ABBR',
 'COUNTY',
 'LOCATION',
 'AREA_SQMI',
 'E_TOTPOP',
 'M_TOTPOP',
 'E_HU',
 'M_HU',
 'E_HH',
 'M_HH',
 'E_POV',
 'M_POV',
 'E_UNEMP',
 'M_UNEMP',
 'E_PCI',
 'M_PCI',
 'E_NOHSDP',
 'M_NOHSDP',
 'E_AGE65',
 'M_AGE65',
 'E_AGE17',
 'M_AGE17',
 'E_DISABL',
 'M_DISABL',
 'E_SNGPNT',
 'M_SNGPNT',
 'E_MINRTY',
 'M_MINRTY',
 'E_LIMENG',
 'M_LIMENG',
 'E_MUNIT',
 'M_MUNIT',
 'E_MOBILE',
 'M_MOBILE',
 'E_CROWD',
 'M_CROWD',
 'E_NOVEH',
 'M_NOVEH',
 'E_GROUPQ',
 'M_GROUPQ',
 'EP_POV',
 'MP_POV',
 'EP_UNEMP',
 'MP_UNEMP',
 'EP_PCI',
 'MP_PCI',
 'EP_NOHSDP',
 'MP_NOHSDP',
 'EP_AGE65',
 'MP_AGE65',
 'EP_AGE17',
 'MP_AGE17',
 'EP_DISABL',
 'MP_DISABL',
 'EP_SNGPNT',
 'MP_SNGPNT',
 'EP_MINRTY',
 'MP_MINRTY',
 'EP_LIMENG',
 'MP_LIMENG',
 'EP_MUNIT',
 'MP_MUNIT',
 'EP_MOBILE',
 'MP_MOBILE',
 'EP_CROWD',
 'MP_CROWD',
 'EP_NOVEH',
 'MP_NOVEH',
 'EP_GROUPQ',
 'MP_GROUPQ',
 'EPL_POV',
 'EPL_UNEMP',
 'EPL_PCI',
 'EPL_NOHSDP',
 'SPL_THEME1',
 'RPL_THEME1',
 'EPL_AGE65',
 'EPL_AGE17'

For this analysis, I decided to drop the following features:
- margin of error
- raw values before controlling for population
- percentile rankings for each variable (instead, opted to use the percentages)

In [20]:
# drop margin of error columns for this analysis
svi_drops = ['ST', 'LOCATION', 'M_TOTPOP',  'M_HU', 'M_HH', 'M_POV', 'M_UNEMP', 'M_PCI', 'M_NOHSDP',
             'M_AGE65', 'M_AGE17', 'M_DISABL', 'M_SNGPNT', 'M_MINRTY', 'M_LIMENG', 'M_MUNIT',
             'M_MOBILE', 'M_CROWD', 'M_NOVEH', 'M_GROUPQ', 'MP_POV', 'MP_UNEMP', 'MP_PCI',
             'MP_NOHSDP', 'MP_AGE65', 'MP_AGE17', 'MP_DISABL', 'MP_SNGPNT', 'MP_MINRTY', 'MP_LIMENG',
             'MP_MUNIT', 'MP_MOBILE', 'MP_CROWD', 'MP_NOVEH', 'MP_GROUPQ',
             'EP_PCI', #EP_PCI is equal to E_PCI
             'E_POV', 'E_UNEMP', 'E_NOHSDP', 'E_AGE65', #dropping values that aren't population controlled
             'E_AGE17', 'E_DISABL', 'E_SNGPNT', 'E_MINRTY', 'E_LIMENG', 'E_MUNIT', 'E_MOBILE',
             'E_CROWD', 'E_NOVEH', 'E_GROUPQ',
             'E_UNINSUR', 'M_UNINSUR', 'MP_UNINSUR', 'E_DAYPOP',
             'EPL_POV', 'EPL_UNEMP', 'EPL_PCI', 'EPL_NOHSDP', 'SPL_THEME1', 'EPL_AGE65', 'EPL_AGE17',
             'EPL_DISABL', 'EPL_SNGPNT', 'SPL_THEME2', 'EPL_MINRTY', 'EPL_LIMENG', 'SPL_THEME3',
             'EPL_MUNIT', 'EPL_MOBILE', 'EPL_CROWD', 'EPL_NOVEH', 'EPL_GROUPQ', 'SPL_THEME4',
             'SPL_THEMES', 'F_POV', 'F_UNEMP', 'F_PCI', 'F_NOHSDP', 'F_THEME1', 'F_AGE65',
             'F_AGE17', 'F_DISABL', 'F_SNGPNT', 'F_THEME2', 'F_MINRTY', 'F_LIMENG', 'F_THEME3',
             'F_MUNIT', 'F_MOBILE', 'F_CROWD', 'F_NOVEH', 'F_GROUPQ', 'F_THEME4', 'F_TOTAL']
svi_mapper = {'E_TOTPOP': 'population_est',
              'E_HU': 'housing_units_est',
              'E_HH': 'households_est',
              'E_PCI': 'per_capita_income_est',
              'EP_POV': 'poverty_rate',
              'EP_UNEMP': 'unemployment_rate',
              'EP_NOHSDP': 'percent_adults_without_hs_diploma', #out of adults 25+
              'EP_AGE65': 'percent_pop_over_65',
              'EP_AGE17': 'percent_pop_under_17',
              'EP_DISABL': 'percent_pop_disability',
              'EP_SNGPNT': 'percent_households_single_parent', #with children under 18
              'EP_MINRTY': 'percent_pop_minority',
              'EP_LIMENG': 'percent_pop_limited_english', #persons 5+ who speak english "less than well"
              'EP_MUNIT': 'percentage_housing_multiunit', #percentage of housing in structures with 10+ units
              'EP_MOBILE': 'percentage_housing_mobile', #percentage of housing in mobile homes
              'EP_CROWD': 'percentage_housing_crowded', #percentage of households with more people than rooms
              'EP_NOVEH': 'percentage_households_without_vehicle',
              'EP_GROUPQ': 'percent_pop_in_group_quarters',
              'EP_UNINSUR': 'percent_pop_uninsured',
              'RPL_THEME1': 'socioeconomic_percentile_ranking',
              'RPL_THEME2': 'household_composition_and_disability_percentile_ranking',
              'RPL_THEME3': 'minority_status_and_language_percentile_ranking',
              'RPL_THEME4': 'housing_type_and_transportation_percentile_ranking',
              'RPL_THEMES': 'overall_percentil_ranking'}

In [21]:
svi.drop(columns=svi_drops, inplace=True)

In [22]:
svi.rename(axis=1, mapper=svi_mapper, inplace=True)

In [23]:
svi

,STATE,ST_ABBR,COUNTY,AREA_SQMI,population_est,housing_units_est,households_est,per_capita_income_est,poverty_rate,unemployment_rate,percent_adults_without_hs_diploma,percent_pop_over_65,percent_pop_under_17,percent_pop_disability,percent_households_single_parent,percent_pop_minority,percent_pop_limited_english,percentage_housing_multiunit,percentage_housing_mobile,percentage_housing_crowded,percentage_households_without_vehicle,percent_pop_in_group_quarters,socioeconomic_percentile_ranking,household_composition_and_disability_percentile_ranking,minority_status_and_language_percentile_ranking,housing_type_and_transportation_percentile_ranking,overall_percentil_ranking,percent_pop_uninsured
FIPS,,,,,,,,,,,,,,,,,,,,,,,,,,,,
35039,NEW MEXICO,NM,Rio Arriba,5860.869195,39307,20044,12398,-999,-999.0,-999.0,13.8,18.0,23.7,16.1,10.7,87.5,2.1,0.3,38.8,2.1,6.2,1.7,-999.0000,0.8758,0.9268,0.5409,-999.0000,10.6
1001,ALABAMA,AL,Autauga,594.443459,55200,23315,21115,29372,15.4,4.2,11.3,14.6,24.2,19.3,7.5,25.0,0.8,3.8,18.4,1.4,5.6,1.0,0.3631,0.5810,0.5947,0.3741,0.4354,7.1
1009,ALABAMA,AL,Blount,644.830460,57645,24222,20600,22656,14.4,4.1,19.8,17.8,23.4,14.2,7.0,12.9,1.7,0.9,25.2,1.6,4.2,0.9,0.6143,0.3187,0.5915,0.1741,0.4242,11.0
1013,ALABAMA,AL,Butler,776.838201,20025,10026,6708,20430,23.5,6.7,15.4,19.0,22.8,17.7,10.5,48.1,0.5,1.3,26.2,1.8,7.8,1.6,0.8455,0.9077,0.6447,0.6259,0.8653,10.2
1015,ALABAMA,AL,Calhoun,605.867251,115098,53682,45033,24706,18.6,8.8,15.9,16.8,21.9,20.8,10.4,27.5,1.0,3.7,14.7,1.7,5.8,2.7,0.7866,0.8303,0.6507,0.6492,0.8252,9.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48131,TEXAS,TX,Duval,1793.476183,11355,5592,3511,17864,25.6,10.6,32.9,17.8,26.1,26.2,15.8,90.9,5.7,0.5,13.0,7.4,10.0,4.4,0.9825,0.9990,0.9809,0.8978,0.9994,24.5
48505,TEXAS,TX,Zapata,998.411980,14369,6388,4405,17228,39.5,11.0,40.0,13.9,33.6,17.3,17.1,95.7,24.4,0.6,34.2,18.1,7.9,0.2,0.9962,0.9452,0.9990,0.6966,0.9895,29.3
48507,TEXAS,TX,Zavala,1297.406535,12131,4344,3509,13350,34.8,8.4,38.0,13.7,29.5,23.3,16.1,95.0,8.0,1.7,22.8,5.6,14.4,1.6,0.9847,0.9857,0.9917,0.9331,0.9987,18.4


Engineer a feature called "population_density" that represents the number of people per square mile

In [27]:
svi['population_density'] = svi['population_est'] / svi['AREA_SQMI']

In [28]:
svi

,STATE,ST_ABBR,COUNTY,AREA_SQMI,population_est,housing_units_est,households_est,per_capita_income_est,poverty_rate,unemployment_rate,percent_adults_without_hs_diploma,percent_pop_over_65,percent_pop_under_17,percent_pop_disability,percent_households_single_parent,percent_pop_minority,percent_pop_limited_english,percentage_housing_multiunit,percentage_housing_mobile,percentage_housing_crowded,percentage_households_without_vehicle,percent_pop_in_group_quarters,socioeconomic_percentile_ranking,household_composition_and_disability_percentile_ranking,minority_status_and_language_percentile_ranking,housing_type_and_transportation_percentile_ranking,overall_percentil_ranking,percent_pop_uninsured,population_density
FIPS,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
35039,NEW MEXICO,NM,Rio Arriba,5860.869195,39307,20044,12398,-999,-999.0,-999.0,13.8,18.0,23.7,16.1,10.7,87.5,2.1,0.3,38.8,2.1,6.2,1.7,-999.0000,0.8758,0.9268,0.5409,-999.0000,10.6,6.706684
1001,ALABAMA,AL,Autauga,594.443459,55200,23315,21115,29372,15.4,4.2,11.3,14.6,24.2,19.3,7.5,25.0,0.8,3.8,18.4,1.4,5.6,1.0,0.3631,0.5810,0.5947,0.3741,0.4354,7.1,92.859967
1009,ALABAMA,AL,Blount,644.830460,57645,24222,20600,22656,14.4,4.1,19.8,17.8,23.4,14.2,7.0,12.9,1.7,0.9,25.2,1.6,4.2,0.9,0.6143,0.3187,0.5915,0.1741,0.4242,11.0,89.395591
1013,ALABAMA,AL,Butler,776.838201,20025,10026,6708,20430,23.5,6.7,15.4,19.0,22.8,17.7,10.5,48.1,0.5,1.3,26.2,1.8,7.8,1.6,0.8455,0.9077,0.6447,0.6259,0.8653,10.2,25.777569
1015,ALABAMA,AL,Calhoun,605.867251,115098,53682,45033,24706,18.6,8.8,15.9,16.8,21.9,20.8,10.4,27.5,1.0,3.7,14.7,1.7,5.8,2.7,0.7866,0.8303,0.6507,0.6492,0.8252,9.4,189.972308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48131,TEXAS,TX,Duval,1793.476183,11355,5592,3511,17864,25.6,10.6,32.9,17.8,26.1,26.2,15.8,90.9,5.7,0.5,13.0,7.4,10.0,4.4,0.9825,0.9990,0.9809,0.8978,0.9994,24.5,6.331280
48505,TEXAS,TX,Zapata,998.411980,14369,6388,4405,17228,39.5,11.0,40.0,13.9,33.6,17.3,17.1,95.7,24.4,0.6,34.2,18.1,7.9,0.2,0.9962,0.9452,0.9990,0.6966,0.9895,29.3,14.391855
48507,TEXAS,TX,Zavala,1297.406535,12131,4344,3509,13350,34.8,8.4,38.0,13.7,29.5,23.3,16.1,95.0,8.0,1.7,22.8,5.6,14.4,1.6,0.9847,0.9857,0.9917,0.9331,0.9987,18.4,9.350192


____

**VACCINE HESITANCY**

*Source: [US Dept of Health & Human Services ASPE](https://aspe.hhs.gov/pdf-report/vaccine-hesitancy)*

To support state and local communication and outreach efforts, ASPE developed state, county, and sub-state level predictions of hesitancy rates using the U.S. Census Bureau’s [Household Pulse Survey (HPS)](https://www.census.gov/programs-surveys/household-pulse-survey.html). The HPS is nationally representative and includes information on U.S. residents’ intentions to receive the COVID-19 vaccine when available, as well as other sociodemographic and geographic (state, region and metropolitan statistical areas) information. 

In [29]:
vax_hesitancy

,FIPS Code,County Name,State,Estimated hesitant,Estimated hesitant or unsure,Estimated strongly hesitant,Social Vulnerability Index (SVI),SVI Category,CVAC level of concern for vaccination rollout,CVAC Level Of Concern,Percent adults fully vaccinated against COVID-19 (as of 6/10/21),Percent Hispanic,Percent non-Hispanic American Indian/Alaska Native,Percent non-Hispanic Asian,Percent non-Hispanic Black,Percent non-Hispanic Native Hawaiian/Pacific Islander,Percent non-Hispanic White,Geographical Point,State Code,County Boundary,State Boundary
0,1123,"Tallapoosa County, Alabama",ALABAMA,0.1806,0.2400,0.1383,0.89,Very High Vulnerability,0.64,High Concern,0.305,0.0242,0.0022,0.0036,0.2697,0.0000,0.6887,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-85.841259 33.104456, -85.8409...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
1,1121,"Talladega County, Alabama",ALABAMA,0.1783,0.2350,0.1368,0.87,Very High Vulnerability,0.84,Very High Concern,0.265,0.0229,0.0043,0.0061,0.3237,0.0003,0.6263,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-86.303069 33.46316, -86.30306...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
2,1131,"Wilcox County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.93,Very High Vulnerability,0.94,Very High Concern,0.394,0.0053,0.0009,0.0003,0.6938,0.0000,0.2684,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-87.52534299999999 32.132773, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
3,1129,"Washington County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.73,High Vulnerability,0.82,Very High Concern,0.308,0.0146,0.0731,0.0025,0.2354,0.0000,0.6495,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-88.45317899999999 31.505388, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
4,1133,"Winston County, Alabama",ALABAMA,0.1805,0.2313,0.1379,0.70,High Vulnerability,0.80,High Concern,0.163,0.0315,0.0034,0.0016,0.0073,0.0005,0.9370,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-87.63656399999999 34.120908, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,55079,"Milwaukee County, Wisconsin",WISCONSIN,0.1505,0.2144,0.0880,0.81,Very High Vulnerability,0.30,Low Concern,0.543,0.1500,0.0047,0.0428,0.2606,0.0002,0.5124,POINT (-89.732969 44.639954),WI,"MULTIPOLYGON (((-87.994171 43.19255, -87.99331...","MULTIPOLYGON (((-92.887067 45.644148, -92.8825..."
3138,55121,"Trempealeau County, Wisconsin",WISCONSIN,0.1611,0.2126,0.0959,0.28,Low Vulnerability,0.31,Low Concern,0.612,0.0840,0.0034,0.0043,0.0051,0.0000,0.8953,POINT (-89.732969 44.639954),WI,"MULTIPOLYGON (((-91.529034 44.422101, -91.5292...","MULTIPOLYGON (((-92.887067 45.644148, -92.8825..."
3139,56001,"Albany County, Wyoming",WYOMING,0.1949,0.2673,0.1405,0.25,Low Vulnerability,0.63,High Concern,0.445,0.0953,0.0091,0.0327,0.0150,0.0003,0.8248,POINT (-107.55145 42.999627),WY,MULTIPOLYGON (((-106.071399 41.530719999999995...,"MULTIPOLYGON (((-111.044893 43.315719, -111.04..."
3140,55067,"Langlade County, Wisconsin",WISCONSIN,0.1518,0.2007,0.0906,0.35,Low Vulnerability,0.19,Very Low Concern,0.481,0.0197,0.0069,0.0022,0.0125,0.0002,0.9383,POINT (-89.732969 44.639954),WI,"MULTIPOLYGON (((-89.42472 45.293175, -89.42473...","MULTIPOLYGON (((-92.887067 45.644148, -92.8825..."


# Appendix